# NeoLook Scraper
* 2022.07.19a


## Step03: HTML 문서 분석하기

### 저장해 둔 Archive List 읽기

In [1]:
def readList(fileName):
    with open(fileName, 'r') as f: 
        lines = []  
        for line in f:
            #print(line)
            lines.append(line.rstrip())
        return lines
    print(len(lines)) 
       
list = readList("archivesList/total.txt")
print(len(list))
print(list[-10:])

47154
['20220629f', '20220630a', '20220630b', '20220630c', '20220630d', '20220630e', '20220630f', '20220630g', '20220630h', '20220630i']


In [2]:
def getHtmlText(fileName):
    dir = fileName[:4]
    file = fileName[4:]
    path = "htmls/" + dir + "/" + fileName + ".htm"
    print(path)
    with open(path, 'r', encoding='utf-8') as f: 
        return f.read()


text = getHtmlText(list[0])
#print(text)

#testList = ['20220602a', '20220602c', '20220603i', '20220601b'] #개인전, 그룹전, 그룹전, 저널

htmls/1989/19891106a.htm


In [5]:
! pip install --upgrade pip
! pip install bs4
from bs4 import BeautifulSoup as bs
import re
import random # debug 용

  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [6]:
xtra = ["개인", "회화", "조각", "사진", "영상", "설치", "수묵", "서예", "드로잉", "판화", "도예"]
xtraDic = {"개인":"solo", "회화":"painting", "조각":"sculpture", "사진":"photography", "영상":"video", "설치":"installation", "수묵":"painting", "서예":"calligrapht", "드로잉":"drawing", "판화":"printing", "도예":"ceramic"}

def getTypeGenre(whichVolumePage):
    soup = bs(whichVolumePage, "html.parser")
    volumeTag = soup.find(class_="tag") #페이지 하단 하얀 박스에 표기된 정보
    volumeTag = volumeTag.contents[2].strip()
    print(volumeTag)

    type = ""
    genre = ""
    #if("展" in volumeTag):
    if(("展" in volumeTag or volumeTag[-1] == "전" or volumeTag[-1] == "제") and not ("조각회展" in volumeTag)):
        #volumeTag = volumeTag.split("/")
        if(len(volumeTag.split("/"))>1):
            type = "solo"
            #print("solo")
            genre = volumeTag.split("/")[-1].strip()            
        elif(len(volumeTag.split(" "))>=2 and (volumeTag.split(" ")[1][-1] == "展")):
            #print("xtra?")
            type = "group"
            #print("xtra group")
            genre = "group"           
            for item in xtra:
                last = volumeTag.split(" ")[-1]
                #print("item:", item, ", last:", last)
                if(item in last):
                    type = "solo"
                    #print("xtra solo")
                    #genre = volumeTag.split(" ")[1]
                    #genre = genre[:-1]
                    key = item
                    #print("key =", key)
                    genre = xtraDic[key]
                    break              
                '''
                else:
                    type = "group"
                    print("xtra group")
                    genre = "group"
                    break
                '''
        else:
            type = "group"
            genre = "group"
            print("group")
    else:
        print("NOT exhibition") 
        type = "NA" 
        genre = "NA"
    #print("type:", type)    
    #print("genre:", genre)
    return type, genre

htmlText = getHtmlText(list[random.randrange(0, 10000)])
type, genre = getTypeGenre(htmlText)
print("type =", type, ", genre =", genre)

htmls/2008/20081120e.htm
김난영展 / KIMNANYOUNG / 金蘭榮 / painting
type = solo , genre = painting


In [11]:
def getTitle(whichVolumePage):
    soup = bs(whichVolumePage, "html.parser")
    volumeTag = soup.find(class_="tag") #페이지 하단 하얀 박스에 표기된 정보
    volumeTag = volumeTag.contents[2].strip()
    print("volumeTag:", volumeTag)
    title = ""
    h1 = soup.find("h1")
    h2 = soup.find("h2")
    if(h1 != None):
        title = h1.text.strip()        
    else:
        title = h2.text.strip()
    print("title:",title)  
    return title

htmlText = getHtmlText(list[random.randrange(0, 10000)])
title = getTitle(htmlText)

htmls/2008/20081008f.htm
volumeTag: 김보라展 / KIMBORA / 金보라 / sculpture
title: dreaming - 꿈으로 피어나다


In [12]:
#import re

def getSoloArtist(whichVolumePage):
    soup = bs(whichVolumePage, "html.parser")
    volumeTag = soup.find(class_="tag") #페이지 하단 하얀 박스에 표기된 정보
    volumeTag = volumeTag.contents[2].strip()
    print(volumeTag)

    artists = []
    tags = volumeTag.split("/")    
    if(len(tags) >= 2 and tags[0].strip()[-1] == "展"):
        print("parsed")
        artist = tags[0].strip()[:-1]
        artists.append(artist)
        #print(artists) 
    
    tags = volumeTag.split(" ")  
    if(len(tags) >= 2 and tags[-1][-1].strip() == "展"):
        artist = tags[0].strip()
        artists.append(artist)
        print(tags[-1]) 
    
    return artists

htmlText = getHtmlText(list[random.randrange(0, 10000)])
artists = getSoloArtist(htmlText)
print(artists)

htmls/2004/20040909a.htm
김보중 회화展
회화展
['김보중']


In [13]:
#import re

def getGroupArtist(whichVolumePage):
    soup = bs(whichVolumePage, "html.parser")
    volumeTag = soup.find(class_="tag") #페이지 하단 하얀 박스에 표기된 정보
    volumeTag = volumeTag.contents[2].strip()
    print(volumeTag)

    artists = []
    description = soup.find(class_="archives-description")
    dd = description.find_all("dd")
    #print(len(dd)) #캡션의 수 출력
    for child in dd:
        #print(child.text)
        if(len(child.text) > 1):
            artists.append(re.split(" |_", child.text.strip())[0])
    print(artists)
    return artists

htmlText = getHtmlText(list[random.randrange(0, 10000)])
artists = getGroupArtist(htmlText)

htmls/2007/20070825d.htm
T . T CLUSTER展
['김경옥-잘림', '김은선-그림만들기', '김채린-열아홉개의', '박정흠-', '유민선', '유태준-3개의애드립', '이정진-그림을', '이주현-']


In [14]:
#import re
extraGenre = ["개인", "화화", "조각", "목조각", "사진", "영상", "사진.영상", "설치영상"]

def getGenre(whichVolumePage):
    soup = bs(whichVolumePage, "html.parser")
    volumeTag = soup.find(class_="tag") #페이지 하단 하얀 박스에 표기된 정보
    volumeTag = volumeTag.contents[2].strip()
    print(volumeTag)

    genre = ""
    if("展" in volumeTag or volumeTag[-1] == "전"):
        #isExhibition = True
        #print("exhibition")
        tag = volumeTag.split("/")
        #tag = re.split(" |/", volumeTag.strip())
        if(len(tag)>1):
            genre = tag[-1].strip()            
        else:
            #print(tag.contents[2].strip())
            for item in extraGenre:
                #print(item)
                
                if(item in volumeTag):
                    print("extra:", item)
                    genre = item 
                #break                             
    else:
        genre = "NA" 
        
    if("공모" in volumeTag):
        genre = "NA"
    print(genre)
    return genre


htmlText = getHtmlText(list[random.randrange(0, 10000)])
genre = getGenre(htmlText)

htmls/2006/20060314d.htm
한국 근대 사진의 선각자들 1부展
extra: 사진
사진


In [17]:
import re

def replaceSrc(imgDesc) :
    descIndex = ["artist", "title", "media", "size", "year"]
    replacedDesc = imgDesc
    for i in range(len(replacedDesc), 5) :
        replacedDesc.append("")
    if replacedDesc[2].find("cm") != -1 and replacedDesc[2].find("×") != -1:
        replacedDesc[4] = replacedDesc[3]
        replacedDesc[3] = replacedDesc[2]
        replacedDesc[2] = ""
    for i in range(len(replacedDesc)) :
        if replacedDesc[i] == "" :
            replacedDesc[i] = f"{descIndex[i]} : NA"
    return replacedDesc

def getImgData(whichVolumePage):
    soup = bs(whichVolumePage, "html.parser")
 
    description = soup.find(class_="archives-description")
    dd = description.find_all("dd")
    dt = description.find_all("dt")
    
    filteredText = []
    imgSrcs = soup.find_all('dt')
    imgDesc = soup.find_all('dd')
    #imgSrcs[0] and imgDesc[0] = 통일전망대 이미지 고정
    for i in range(1, len(imgSrcs)):
        try:
            imgSrcs[i] = imgSrcs[i].find('img')['src']
            imgDesc[i] = imgDesc[i].get_text().replace(',', '').replace('\n', '').strip().replace('\xa0', '')
            imgDesc[i] = imgDesc[i].split('_')
            imgDesc[i] = replaceSrc(imgDesc[i])
            filteredText.append([imgDesc[i], imgSrcs[i]])
        except Exception as e:
            print("error ", imgSrcs[i], e)
            continue

    return imgSrcs, imgDesc, filteredText

testList = ['19891106b','20000120a','19991207a','19991005a','19970303a','19931126a','19930209a','19921013a','19921002a','19920901b', '19920505a', '19910309a', '20220602a', '20220602c', '20220603i', '20220601b'] #개인전, 그룹전, 그룹전, 저널
htmlText = getHtmlText(testList[0])
#print(htmlText)
imgSrcs, imgDesc, filteredText = getImgData(htmlText)
print(*filteredText, sep="\n")
print("")

htmls/1989/19891106b.htm
[['이인철', '스포츠 공화국의 上下', '종이에 목판화', 'size : NA', 'year : NA'], 'https://neolook.com/archives/1989110601b.jpg']
[['이인철', '역사의 기록', '종이에 목판화', 'size : NA', 'year : NA'], 'https://neolook.com/archives/1989110602b.jpg']
[['이인철', '대장정', '종이에 목판화', 'size : NA', 'year : NA'], 'https://neolook.com/archives/1989110603b.jpg']
[['이인철', '반전 반핵', '종이에 목판화', 'size : NA', 'year : NA'], 'https://neolook.com/archives/1989110604b.jpg']
[['이인철', '머물지 못하는 사람들', '종이에 목판화', 'size : NA', 'year : NA'], 'https://neolook.com/archives/1989110605b.jpg']
[['이인철', '하늘아 열려라', '종이에 목판화', 'size : NA', 'year : NA'], 'https://neolook.com/archives/1989110606b.jpg']
[['이인철', '잃어버린 창', '종이에 목판화', 'size : NA', 'year : NA'], 'https://neolook.com/archives/1989110607b.jpg']
[['이상호', '격려', 'media : NA', '18×20.5cm', '1988'], 'https://neolook.com/archives/1989110608b.jpg']
[['이상호', '슬퍼하는 청년', 'media : NA', '22×22.5cm', '1988'], 'https://neolook.com/archives/1989110609b.jpg']
[['이상호', '기다림', '종이에 목판화', '19

In [69]:
import re


def getDescData(whichVolumePage):
    soup = bs(whichVolumePage, "html.parser")

    description = soup.find(class_="archives-description")
    descText = soup.find_all('p')
    filteredText = ""
    for i in reversed(range(len(descText))):
        text = descText[i]
        try:
            if str(text["class"]).find('line') != -1:
                del descText[i]
        except Exception as e:
            print("error ", e, "index ", i, " not had class")
            continue
    for text in descText:
        text = text.get_text().split('\n')
        text = map(lambda x: x.strip(), text)
        text = filter(lambda x: len(x) > 1, text)
        text = filter(lambda x: x != '\xa0' and x != ' ' , text)
        text = filter(lambda x: x.find("위 이미지를") == -1, text)
        text = '\n'.join(text)
        print(f"\033[95m filtered Text : \033[0m {text} \033[95m***\033[0m")
        filteredText += text + '\n'
    filteredText.replace('\n\n', '\n').replace('\xa0', '')

    return imgSrcs, imgDesc, filteredText

testList = ['19891106b', '20000120a', '19991207a', '19991005a', '19970303a', '19931126a', '19930209a', '19921013a',
            '19921002a', '19920901b', '19920505a', '19910309a', '20220602a', '20220602c', '20220603i', '20220601b']  # 개인전, 그룹전, 그룹전, 저널
htmlText = getHtmlText(testList[1])
#print(htmlText)
imgSrcs, imgDesc, filteredText = getDescData(htmlText)
#print(filteredText)


htmls/2000/20000120a.htm
error  'class' index  6  not had class
 filtered Text :   ***
 filtered Text :  참여작가
김수자_김영진_박홍천_배병우_육근병_이불_최정화 ***
 filtered Text :  아트선재센터
서울 종로구 소격동 144-2 번지
Tel. 02_733_8942 ***
 filtered Text :  "『느림』전은 근대화 과정에서 야기되었던 빠른 일상성에서 벗어나 우리의 속도를 찾고자 기획된 전시이다."
● 아트선재센터가 2000년을 맞아 처음으로 선보이는 전시는 『느림(Slowness of Speed)』이다. 『느림』은 지난 1998년과 1999년 호주 멜버른의 빅토리아 미술관과 시드니의 뉴 사우스 웨일즈 미술관에서 선보여 많은 호응을 얻었던 전시로, 해외 미술관과의 활발한 교류를 통해 우리의 미술과 작가들을 해외에 소개하고 알리는 데에 주력해왔던 아트선재의 활동이 해외에서도 크게 인정을 받게 되었던 계기를 마련해준 전시였다. ● 1998년 7월과 8월 서울 아트선재센터와 경주 아트선재미술관에서 열렸던 호주 현대작가전 『언홈리(Unhomely)』에 대한 교환전으로 기획되었던 이 전시에는 김수자, 김영진, 박홍천, 배병우, 육근병, 이불, 최정화 등 한국의 젊은 작가 7명이 참여하고 있다. ● 『느림』이라는 제목으로 구성된 이 전시는 우리의 근대화 과정에서 단순히 "빠름"으로만 변질되어버린 "속도"의 개념을 재고해보려는 의도에서 기획된 것이다. 사실 우리의 속도는 변화무쌍했던 근대화 과정에서 보여지는 "빠름"이 아니라 오히려 "산보", "소요(逍遙)"에서 유추되는 "느림" 혹은 "유유자적"이 아니었을까. 그러나 서구의 문물을 받아들이고 산업화, 과학화, 현대화를 거치는 동안 "빠름"은 발전과 진보의 다른 이름으로 대체되어 상대적인 가치우위를 점유하게 되었고, 예술이나 문화의 분야에서도 서구의 그것을 빠르게 따라가는 것만이 발전과 진보를 이루는 것이라는 생각이 만